In [12]:
import pandas as pd


data = pd.read_csv('/Users/saideepak/Desktop/Daen 690/untitled folder/cleaned_data.csv')

data_sorted = data.sort_values(by=['stationId', 'date'])
data_sorted['is_hot'] = data_sorted['TMAX'] > 33
data_sorted['is_cold'] = data_sorted['TMIN'] < -10

data_sorted['hot_streak_id'] = (data_sorted['is_hot'] != data_sorted['is_hot'].shift(1)).cumsum()
data_sorted['cold_streak_id'] = (data_sorted['is_cold'] != data_sorted['is_cold'].shift(1)).cumsum()

heatwave_streaks = data_sorted[data_sorted['is_hot']].groupby('hot_streak_id').filter(lambda x: len(x) >= 2)
coldwave_streaks = data_sorted[data_sorted['is_cold']].groupby('cold_streak_id').filter(lambda x: len(x) >= 2)
all_heatwave_days = data_sorted[data_sorted['hot_streak_id'].isin(heatwave_streaks['hot_streak_id'])]
all_coldwave_days = data_sorted[data_sorted['cold_streak_id'].isin(coldwave_streaks['cold_streak_id'])]


data_sorted = data_sorted.merge(all_heatwave_days[['date', 'stationId', 'hot_streak_id']], 
                                on=['date', 'stationId'], 
                                how='left', 
                                indicator='is_in_heatwave')

data_sorted = data_sorted.merge(all_coldwave_days[['date', 'stationId', 'cold_streak_id']], 
                                on=['date', 'stationId'], 
                                how='left', 
                                indicator='is_in_coldwave')


data_sorted['is_heatwave'] = data_sorted['is_in_heatwave'] == 'both'
data_sorted['is_coldwave'] = data_sorted['is_in_coldwave'] == 'both'


data_sorted.drop(['hot_streak_id', 'cold_streak_id', 'is_in_heatwave', 'is_in_coldwave'], axis=1, inplace=True, errors='ignore')


print(data_sorted.head())



   stationId           county        date  PRCP   TAVG   TMAX  TMIN  year  \
0      44001  Accomack County  1951-01-01   0.0   2.17   6.04 -1.71  1951   
1      44001  Accomack County  1951-01-02   0.0   0.59   4.75 -3.58  1951   
2      44001  Accomack County  1951-01-03   0.0   5.19  11.64 -1.25  1951   
3      44001  Accomack County  1951-01-04   0.0  11.00  17.70  4.30  1951   
4      44001  Accomack County  1951-01-05   0.0   8.36  15.38  1.33  1951   

   month  day  is_hot  is_cold  hot_streak_id_x  cold_streak_id_x  \
0      1    1   False    False                1                 1   
1      1    2   False    False                1                 1   
2      1    3   False    False                1                 1   
3      1    4   False    False                1                 1   
4      1    5   False    False                1                 1   

   hot_streak_id_y  cold_streak_id_y  is_heatwave  is_coldwave  
0              NaN               NaN        False        

In [13]:

# Calculate the 5th percentile of the TMIN column
tmin_5th_percentile = data['TMIN'].quantile(0.05)

print(f"The 5th percentile of TMIN is {tmin_5th_percentile}")


The 5th percentile of TMIN is -7.4


In [11]:

# Calculate the 5th percentile of the TMIN column
tmax_95th_percentile = data['TMAX'].quantile(0.95)

print(f"The 95th percentile of TMAX is {tmax_95th_percentile}")

The 95th percentile of TMAX is 32.39


In [14]:
# Save the DataFrame to a new CSV file for use in Tableau
data_sorted.to_csv('/Users/saideepak/Desktop/Daen 690/untitled folder.csv', index=False)